In [36]:
# # Imports and environment

import os
import sys
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI
from markdown_pdf import MarkdownPdf, Section

# Load environment variables (e.g. OPENAI_API_KEY)
load_dotenv()

True

In [37]:
# # User‐configurable variables

# 1. Define the topic you want to learn
TOPIC = "AI Engineering to build ai powered apps and agents"            # e.g. "Apache Iceberg"

# 2. Define how many parts to split into
N_PARTS = 20                    # e.g. 10 parts

# 3. Output PDF configuration
output_file = "../results/AI_Engineering_by_ChatGPT.pdf"
model = "gpt-4.1-mini"
toc_level = 3
optimize = False
use_web_search = True
verbose = True
max_output_tokens = 2000

In [38]:
# # Helper function definitions

def generate_markdown_from_prompt(current_prompt: str,
                                  prev_prompt_results: list[dict],
                                  client) -> tuple[str, list[dict]]:
    """
    - current_prompt: new user instruction ("Now generate part i only.")
    - prev_prompt_results: list of prior messages [{"role": "...", "content": "..."}]
    - client: OpenAI client

    Returns: (assistant_markdown, updated_history_list)
    """
    try:
        # Copy prior history and append the new user message
        messages = prev_prompt_results.copy()
        messages.append({"role": "user", "content": current_prompt})

        # Decide whether to use web-search tool
        tools = [{"type": "web_search_preview"}] if use_web_search else []

        # Call the API with the entire history
        response = client.responses.create(
            model=model,
            tools=tools,
            input=messages,
            max_output_tokens=max_output_tokens
        )

        # Extract only the assistant’s new reply text
        assistant_reply = response.output_text.strip()

        # Append assistant’s message to history so next call has full context
        messages.append({"role": "assistant", "content": assistant_reply})

        # Return the "Prompt: ..." wrapper plus the reply, and the updated history
        return (
            f"Prompt: {current_prompt}\n\n" + assistant_reply,
            messages
        )

    except Exception as e:
        print(f"AI request failed for prompt '{current_prompt}': {e}")
        # Even on failure, return the unmodified history so we don’t lose context
        return f"{current_prompt}\n\n---\n\n", prev_prompt_results

In [39]:
# # Generate all parts with context and build PDF

# Read custom CSS if any
css = Path("custom.css").read_text(encoding="utf-8")
pdf = MarkdownPdf(toc_level=toc_level, optimize=optimize)

# Verify API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("Environment variable OPENAI_API_KEY is not set.")
    sys.exit(1)

# Instantiate client
client = OpenAI(api_key=api_key)

# Initialize history with a single system instruction
history_messages = [
    {
        "role": "system",
        "content": (
            "Provide output in Markdown. "
            "Use **bold** text for headers and do not use '#' headers. "
            "Don't add line separators in the response."
        )
    }
]

# Loop through each part
for part_idx in range(0, N_PARTS + 1):
    if part_idx == 0:
        # First prompt has no prior assistant replies
        current_prompt = f"Teach me {TOPIC} in {N_PARTS} prompts. Now generate the prompts only."
        section = Section("# " + current_prompt)
        pdf.add_section(section, user_css=css)
    else:
        # Subsequent prompts rely on history_messages
        current_prompt = f"Now generate response for prompt # {part_idx} only."

    if verbose:
        print(f"Requesting part {part_idx}/{N_PARTS}…")

    # Call helper, passing current_prompt + accumulated history
    md_content, history_messages = generate_markdown_from_prompt(
        current_prompt,
        history_messages,
        client
    )

    # Add returned Markdown as a new PDF section
    section = Section(md_content)
    pdf.add_section(section, user_css=css)

Requesting part 0/20…
Requesting part 1/20…
Requesting part 2/20…
Requesting part 3/20…
Requesting part 4/20…
Requesting part 5/20…
Requesting part 6/20…
Requesting part 7/20…
Requesting part 8/20…
Requesting part 9/20…
Requesting part 10/20…
Requesting part 11/20…
Requesting part 12/20…
Requesting part 13/20…
Requesting part 14/20…
Requesting part 15/20…
Requesting part 16/20…
Requesting part 17/20…
Requesting part 18/20…
Requesting part 19/20…
Requesting part 20/20…


In [40]:
# # Save as PDF
try:
    pdf.save(output_file)
    print(f"PDF successfully saved to {output_file}")
except Exception as e:
    print(f"Failed to save PDF: {e}")
    raise

PDF successfully saved to ../results/AI_Engineering_by_ChatGPT.pdf
